#### Update of this notebook:
- *IRS Form 990 e-File Data (6a) -- Change Data Types.ipynb*

### Note
These two variables are ``float64`` instead of ``obj`` -- they must all be missing ``['F9_03_PC_PROG_SVC_ACC_2_CODE', 'F9_03_PC_PROG_SVC_ACC_3_CODE']``

# Overview

Main purpose of notebook:
- All of the variables are currently in 'string' format, so in this notebook I change the data type for relevant variables to a *timestamp* or *int* or *float*. Implementing new approach here for changing data types.

Steps:
- Read in *concordance_VERIFIED.xlsx* in order to access the *python_data_type* column
    - Collapse to *new_variables_df* then use that DF

- Read in e-file DF: 
    - *all filings may 2021 - all control variables (with parsed sub-key variables).pkl.gz*

- Change data types for *DateTime* and *Int64* variables:
    - string variables -- nothing done
	- DateTime variables -- change both relevant variables
	- Int64 variables -- change all with one-liner: 
	    - df[Int64_vars] = df[Int64_vars].apply(pd.to_numeric)
        - This one-liner averts issues with converting to 'Int64' -- it chooses either 'Int64' or 'float'

- Save DF:
	- *all filings may 2021 - all control variables (with parsed sub-key variables and reformatted types).pkl.gz*

Note:
- Even though I use 'Int64' as the value for all integer columns in *python_data_type*, the *pd.numeric( )* code chooses whether to convert to 'Int64' or 'float'	
- Three variables -- *fiscal_year*,  *TaxPeriod*, and *F9_00_HD_TAX_PER_END* -- are all based off the same date, which is the *END* of the tax period, while *F9_00_HD_TAX_YEAR* reflects the year in which the tax period *BEGINS*. See *IRS 990 e-File Data -- CONTROL VARIABLES (A2) -- Combine Columns (Python 3.6).ipynb*
- In a previous version of this notebook I looped over all variables in each *data_type_xsd* (e.g., DateType, StateType, etc.) and inspected the *dtypes* and sample values for these variables. Based on that I added the *python_data_type* column to the concordance file:
    - See *IRS 990 e-File Data -- CONTROL VARIABLES (A4-1a) -- Change Data Types (with data type verification code -- no longer needed).ipynb*

# Load Packages and Connect to MongoDB

In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
from pandas import Series

In [2]:
print(pd.__version__)

2.2.2


In [3]:
#http://pandas.pydata.org/pandas-docs/stable/options.html
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 250)

In [4]:
import datetime

In [10]:
import gc

#### Set working directory

In [6]:
cd "C:\\Users\\Gregory\\IRS 990 Control Variables\\"

C:\Users\Gregory\IRS 990 Control Variables


# Read in Concordance File
We are going to read in the 'concordance' file, which is a specialized codebook and contains details about each data column in the e-file data.

In [7]:
%%time
print ("Current date and time : ", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), '\n')
concordance = pd.read_excel('concordance_VERIFIED.xlsx')
print('# of columns:', len(concordance.columns))
print('# of observations:', len(concordance))
concordance[:2]

Current date and time :  2025-04-18 12:12:31 

# of columns: 17
# of observations: 574
CPU times: total: 531 ms
Wall time: 2.32 s


,xpath,variable_name_new,# of Characters (newly named),variable name notes,PARSING NOTES,OTHER NOTES,description,location_code,part,data_type_xsd,python_data_type,fill_null,BINARIZE,MongoDB_Name,sub_key,sub_sub_key,cardinality
0,/Return/ReturnData/IRS990/SpecialConditionDesc,F9_00_HD_SPECIAL_CONDITION_DESC,NaN,NaN,NaN,NaN,Special condition description,F990-PC-PART-00,PART-00,TextType,string,Do not fill null,NaN,SpecialConditionDesc,NaN,NaN,NaN
1,/Return/ReturnData/IRS990/SpecialConditionDescription,F9_00_HD_SPECIAL_CONDITION_DESC,31.0,NaN,NaN,NaN,Special condition description,F990-PC-PART-00,PART-00,TextType,string,Do not fill null,NaN,SpecialConditionDescription,NaN,NaN,NaN


In [8]:
concordance[concordance['sub_key'].notnull()][['variable_name_new', 'MongoDB_Name', 'sub_key']]

,variable_name_new,MongoDB_Name,sub_key
122,F9_03_PC_PROG_SVC_ACC_2_CODE,Activity2,ActivityCode
123,F9_03_PC_PROG_SVC_ACC_2_CODE,ProgSrvcAccomActy2Grp,ActivityCode
124,F9_03_PC_PROG_SVC_ACC_3_CODE,Activity3,ActivityCode
125,F9_03_PC_PROG_SVC_ACC_3_CODE,ProgSrvcAccomActy3Grp,ActivityCode
146,F9_03_PC_PROG_SVC_ACC_2_DESC,Activity2,Description
...,...,...,...
556,F9_00_HD_FILER_CITY_US,Filer,USAddress
557,F9_00_HD_FILER_COUNTRY_FRGN,Filer,ForeignAddress
558,F9_00_HD_FILER_COUNTRY_FRGN,Filer,ForeignAddress
559,F9_00_HD_FILER_ZIP_US,Filer,USAddress


# Read 990 DB into PANDAS DF
Read in the PANDAS dataframe from the previous notebook.

In [9]:
#%%time
#import datetime
#print ("Current date and time : ", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), '\n')
#df = pd.read_pickle('all NEW filings February 2024 - all control variables (with parsed sub-key variables).pkl.gz', compression='gzip')
#print('# of columns:', len(df.columns))
#print('# of observations:', len(df))
#df[:1]

In [12]:
%%time
import datetime
print ("Current date and time : ", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), '\n')
df = pd.read_feather('D:/all_filings_april_2025_all_controls_combined_parsed.feather')
print('# of columns:', len(df.columns))
print('# of observations:', len(df))
df[:1]

Current date and time :  2025-04-18 12:13:09 

# of columns: 305
# of observations: 3469008
CPU times: total: 2min 43s
Wall time: 2min 19s


,_id,OrganizationName,URL,DLN,TaxPeriod,F9_09_PC_FEES_FOR_SVCE_FR_TOT,F9_00_HD_BUILD_TIME_STAMP,fiscal_year,EIN,Name,NameControl,Phone,USAddress,ForeignAddress,InCareOfName,BusinessName,BusinessNameControlTxt,PhoneNum,InCareOfNm,ForeignPhoneNum,F9_00_HD_ADDR_CHANGE,F9_00_HD_AMENDED_RETURN,F9_00_HD_CTRY_OF_DOMICILE,F9_00_HD_EXEMPT_STATUS_4847A1,F9_00_HD_EXEMPT_STATUS_501C,F9_00_HD_EXEMPT_STATUS_501C3,F9_00_HD_FINAL_RETURN,F9_00_HD_GROSS_EXEMPT_NUM,F9_00_HD_GROSS_RCPT,F9_00_HD_GROUP_RETURN,F9_00_HD_INCLUDES_SUBORD_ORGS,F9_00_HD_INITIAL_RETURN,F9_00_HD_PRIN_OFF_NAME,F9_00_HD_SIGNING_OFFICER_SIGNTR,F9_00_HD_SPECIAL_CONDITION_DESC,F9_00_HD_STATE_OF_DOMICILE,F9_00_HD_TAX_PER_BEGIN,F9_00_HD_TAX_PER_END,F9_00_HD_TAX_YEAR,F9_00_HD_TIME_STAMP,F9_00_HD_TYPE_ORG_ASSOCIATION,F9_00_HD_TYPE_ORG_CORP,F9_00_HD_TYPE_ORG_OTHER,F9_00_HD_TYPE_ORG_OTHER_DESC,F9_00_HD_TYPE_ORG_TRUST,F9_00_HD_WEBSITE,F9_00_HD_YEAR_FORMED,F9_01_PC_BEN_PAID_MEMB_PRIOR,F9_01_PC_CONTR_GRANTS_CURR,F9_01_PC_CONTR_GRANTS_PRIOR,F9_01_PC_GRANTS_PRIOR,F9_01_PC_INDEP_VOTING_MEMB,F9_01_PC_INVEST_INCOME_PRIOR,F9_01_PC_NET_ASSETS_BOY,F9_01_PC_OTHER_EXPENSE_PRIOR,F9_01_PC_OTHER_REV_PRIOR,F9_01_PC_PROF_FUNDRISING_EXP_CURR,F9_01_PC_PROF_FUNDRISING_EXP_PRIOR,F9_01_PC_PROG_SERVICE_REV_PRIOR,F9_01_PC_REV_LESS_EXP_CURR,F9_01_PC_REV_LESS_EXP_PRIOR,F9_01_PC_TERMINATION_CONTRACTION,F9_01_PC_TOT_ASSETS_EOY,F9_01_PC_TOT_EXP_PRIOR,F9_01_PC_TOT_FNDR_EXP_CURR,F9_01_PC_TOT_INDIV_EMPLOYED,F9_01_PC_TOT_INDIV_VOLUNTEERS,F9_01_PC_TOT_LIABILITIES_EOY,F9_01_PC_TOT_REVENUE_PRIOR,F9_01_PC_TOT_UBI_GROSS,F9_01_PC_TOT_UBI_NET,F9_01_PC_VOTING_MEMB_GOV_BODY,F9_01_PZ_BEN_PAID_TO_MEMB_CURR,F9_01_PZ_GRANTS_PAID_CURR,F9_01_PZ_INVEST_INCOME_CURR,F9_01_PZ_NAFB_EOY,F9_01_PZ_ORGANIZATIONAL_MISSION,F9_01_PZ_OTHER_EXPENSE_CURR,F9_01_PZ_OTHER_REV_CURR,F9_01_PZ_PROG_SERVICE_REV_CURR,F9_01_PZ_SALARIES_CURR,F9_01_PZ_SALARIES_PRIOR,F9_01_PZ_TOT_ASSETS_BOY,F9_01_PZ_TOT_EXP_CURR,F9_01_PZ_TOT_LIAB_BOY,F9_01_PZ_TOT_REV_CURR,F9_03_PC_PGMSVC_SIGNIF_CHG,F9_03_PC_PGMSVC_SIGNIF_NEW,F9_03_PC_PROG_SVC_ACC_1_CODE,F9_03_PC_PROG_SVC_ACC_1_DESC,F9_03_PC_PROG_SVC_ACC_1_EXP,F9_03_PC_PROG_SVC_ACC_1_GRNT,F9_03_PC_PROG_SVC_ACC_1_REV,F9_03_PC_PROG_SVC_ACC_2_CODE,F9_03_PC_PROG_SVC_ACC_2_DESC,F9_03_PC_PROG_SVC_ACC_2_EXP,F9_03_PC_PROG_SVC_ACC_2_GRNT,F9_03_PC_PROG_SVC_ACC_2_REV,F9_03_PC_PROG_SVC_ACC_3_CODE,F9_03_PC_PROG_SVC_ACC_3_DESC,F9_03_PC_PROG_SVC_ACC_3_EXP,F9_03_PC_PROG_SVC_ACC_3_GRNT,F9_03_PC_PROG_SVC_ACC_3_REV,F9_03_PC_TOT_OTH_PROG_SVC_EXP,F9_03_PC_TOT_OTH_PROG_SVC_GRNT,F9_03_PC_TOT_OTH_PROG_SVC_REV,F9_03_PC_TOT_PROG_SVC_EXPENSE,F9_03_PZ_MISSION_DESCRIPTION,F9_03_PZ_SCHEDULE_O_PART3,F9_04_PC_ACTVITIES_VIA_PARTNER,F9_04_PC_CONTROLLED_ENTITY,F9_04_PC_DISREGARDED_ENTITY,F9_04_PC_EXCESS_BENEFIT_TRANS,F9_04_PC_FR_EVENT_INC_GT_15K,F9_04_PC_GAMING_INC_GT_15K,F9_04_PC_LOBBYING_ACTIVITIES,F9_04_PC_POLITICAL_ACTIVITIES,F9_04_PC_PRIOR_EXCESS_BEN_TRAN,F9_04_PC_PROF_FR_EXP_GT_15K,F9_04_PC_RELATED_ENTITY,F9_04_PC_TRANS_TO_CNTRLD_ENT,F9_04_PC_TRANS_WITH_CNTRLD_ENT,F9_05_EXP_SCHED_O_X,F9_05_PC_NUMBER_EMPLOYEES_W3,F9_05_PC_NUMBER_FORMS_1096,F9_05_PC_UNRELATED_BUS_INCOME,F9_06_EXP_SCHED_O_X,F9_06_PC_990_PROVIDED_GOV_BODY,F9_06_PC_ANNUAL_DISC_COVRD_PERS,F9_06_PC_CEO_COMPENSTN_PROCESS,F9_06_PC_CHANGES_ORGANIZING_DOCS,F9_06_PC_CONFLICT_OF_INTEREST,F9_06_PC_DECISIONS_SUBJ_APPROVAL,F9_06_PC_DELEGATION_MGT_DUTIES,F9_06_PC_DELEGATION_OF_MGT,F9_06_PC_DOCUMENT_RET_POLICY,F9_06_PC_ELECTION_BOARD_MEMBERS,F9_06_PC_FAMILY_OR_BUSINESS_REL,F9_06_PC_FORM_AVAIL_OWN_WEBSITE,F9_06_PC_FORM_UPON_REQUEST,F9_06_PC_JOINT_VENTURE_INVESTMNT,F9_06_PC_JOINT_VENTURE_POLICY,F9_06_PC_LOCAL_CHAPTERS,F9_06_PC_MATERIAL_DIVERSION,F9_06_PC_MEMBERS_OR_STOCKHOLDERS,F9_06_PC_MINUTES_COMMITTEES,F9_06_PC_MINUTES_GOVERNING_BODY,F9_06_PC_MONITORING_OF_COI_POLICY,F9_06_PC_NUM_IND_VOTING_MEMBERS,F9_06_PC_NUM_VOTING_GOV_MEMBERS,F9_06_PC_OFFICER_MAILING_ADDRESS,F9_06_PC_OTHER_COMPENSTN_PROCESS,F9_06_PC_OTHER_WEBSITE,F9_06_PC_OWN_WEBSITE,F9_06_PC_POLICIES_GOVERN_CHAPTER,F9_06_PC_STATES_WHERE_RET_FILED,F9_

# Combine variables for alternative concordance file
Here we are 'collapsing' the concordance file to have one row per variable.

In [13]:
#def agg_funcs(x):
#    names = {
#        'original_names':  list(set(x['MongoDB_Name'].tolist())),
#        'python_data_type': x['python_data_type'].head(1).values[0]
#    }
#    #THE FOLLOWING SHORTCUT WORKS BUT CHANGES THE ORDER OF THE COLUMNS
#    #return pd.Series(names, index = list(names.keys()))
#    return pd.Series(names, index=['original_names', 'python_data_type'])
#new_variables_df = concordance.groupby(['variable_name_new']).apply(agg_funcs)
#new_variables_df = new_variables_df.reset_index()
#print('# of variables:', len(new_variables_df))
#new_variables_df[:]

# of variables: 288


,variable_name_new,original_names,python_data_type
0,F9_00_HD_ADDR_CHANGE,"[AddressChangeInd, AddressChange]",Int64
1,F9_00_HD_AMENDED_RETURN,"[AmendedReturn, AmendedReturnInd]",Int64
2,F9_00_HD_BUILD_TIME_STAMP,[BuildTS],DateTime
3,F9_00_HD_CTRY_OF_DOMICILE,"[CountryLegalDomicile, LegalDomicileCountryCd]",string
4,F9_00_HD_EXEMPT_STATUS_4847A1,"[Organization4947a1, Organization4947a1NotPFInd]",Int64
...,...,...,...
283,F9_12_PC_FED_GRNT_AUDIT_REQUIRED,"[FederalGrantAuditRequiredInd, FederalGrantAuditRequired]",Int64
284,F9_12_PC_FINCL_STMTS_AUDITED,"[FSAuditedInd, FSAudited]",Int64
285,F9_12_SCHED_O_X,"[InfoInScheduleOPartXII, InfoInScheduleOPartXIIInd]",Int64
286,TaxPeriod,[TaxPeriod],string


In [13]:
%%time
new_variables_df = (
    concordance
    .groupby('variable_name_new')
    .agg(
        original_names=('MongoDB_Name', lambda x: list(set(x))),
        python_data_type=('python_data_type', 'first')
    )
    .reset_index()
  )

print('# of variables:', len(new_variables_df))
new_variables_df

# of variables: 288
CPU times: total: 0 ns
Wall time: 17.5 ms


,variable_name_new,original_names,python_data_type
0,F9_00_HD_ADDR_CHANGE,"[AddressChangeInd, AddressChange]",Int64
1,F9_00_HD_AMENDED_RETURN,"[AmendedReturnInd, AmendedReturn]",Int64
2,F9_00_HD_BUILD_TIME_STAMP,[BuildTS],DateTime
3,F9_00_HD_CTRY_OF_DOMICILE,"[CountryLegalDomicile, LegalDomicileCountryCd]",string
4,F9_00_HD_EXEMPT_STATUS_4847A1,"[Organization4947a1NotPFInd, Organization4947a1]",Int64
...,...,...,...
283,F9_12_PC_FED_GRNT_AUDIT_REQUIRED,"[FederalGrantAuditRequiredInd, FederalGrantAuditRequired]",Int64
284,F9_12_PC_FINCL_STMTS_AUDITED,"[FSAudited, FSAuditedInd]",Int64
285,F9_12_SCHED_O_X,"[InfoInScheduleOPartXIIInd, InfoInScheduleOPartXII]",Int64
286,TaxPeriod,[TaxPeriod],string


In [14]:
df.loc[[695015]]

,_id,OrganizationName,URL,DLN,TaxPeriod,F9_09_PC_FEES_FOR_SVCE_FR_TOT,F9_00_HD_BUILD_TIME_STAMP,fiscal_year,EIN,Name,NameControl,Phone,USAddress,ForeignAddress,InCareOfName,BusinessName,BusinessNameControlTxt,PhoneNum,InCareOfNm,ForeignPhoneNum,F9_00_HD_ADDR_CHANGE,F9_00_HD_AMENDED_RETURN,F9_00_HD_CTRY_OF_DOMICILE,F9_00_HD_EXEMPT_STATUS_4847A1,F9_00_HD_EXEMPT_STATUS_501C,F9_00_HD_EXEMPT_STATUS_501C3,F9_00_HD_FINAL_RETURN,F9_00_HD_GROSS_EXEMPT_NUM,F9_00_HD_GROSS_RCPT,F9_00_HD_GROUP_RETURN,F9_00_HD_INCLUDES_SUBORD_ORGS,F9_00_HD_INITIAL_RETURN,F9_00_HD_PRIN_OFF_NAME,F9_00_HD_SIGNING_OFFICER_SIGNTR,F9_00_HD_SPECIAL_CONDITION_DESC,F9_00_HD_STATE_OF_DOMICILE,F9_00_HD_TAX_PER_BEGIN,F9_00_HD_TAX_PER_END,F9_00_HD_TAX_YEAR,F9_00_HD_TIME_STAMP,F9_00_HD_TYPE_ORG_ASSOCIATION,F9_00_HD_TYPE_ORG_CORP,F9_00_HD_TYPE_ORG_OTHER,F9_00_HD_TYPE_ORG_OTHER_DESC,F9_00_HD_TYPE_ORG_TRUST,F9_00_HD_WEBSITE,F9_00_HD_YEAR_FORMED,F9_01_PC_BEN_PAID_MEMB_PRIOR,F9_01_PC_CONTR_GRANTS_CURR,F9_01_PC_CONTR_GRANTS_PRIOR,F9_01_PC_GRANTS_PRIOR,F9_01_PC_INDEP_VOTING_MEMB,F9_01_PC_INVEST_INCOME_PRIOR,F9_01_PC_NET_ASSETS_BOY,F9_01_PC_OTHER_EXPENSE_PRIOR,F9_01_PC_OTHER_REV_PRIOR,F9_01_PC_PROF_FUNDRISING_EXP_CURR,F9_01_PC_PROF_FUNDRISING_EXP_PRIOR,F9_01_PC_PROG_SERVICE_REV_PRIOR,F9_01_PC_REV_LESS_EXP_CURR,F9_01_PC_REV_LESS_EXP_PRIOR,F9_01_PC_TERMINATION_CONTRACTION,F9_01_PC_TOT_ASSETS_EOY,F9_01_PC_TOT_EXP_PRIOR,F9_01_PC_TOT_FNDR_EXP_CURR,F9_01_PC_TOT_INDIV_EMPLOYED,F9_01_PC_TOT_INDIV_VOLUNTEERS,F9_01_PC_TOT_LIABILITIES_EOY,F9_01_PC_TOT_REVENUE_PRIOR,F9_01_PC_TOT_UBI_GROSS,F9_01_PC_TOT_UBI_NET,F9_01_PC_VOTING_MEMB_GOV_BODY,F9_01_PZ_BEN_PAID_TO_MEMB_CURR,F9_01_PZ_GRANTS_PAID_CURR,F9_01_PZ_INVEST_INCOME_CURR,F9_01_PZ_NAFB_EOY,F9_01_PZ_ORGANIZATIONAL_MISSION,F9_01_PZ_OTHER_EXPENSE_CURR,F9_01_PZ_OTHER_REV_CURR,F9_01_PZ_PROG_SERVICE_REV_CURR,F9_01_PZ_SALARIES_CURR,F9_01_PZ_SALARIES_PRIOR,F9_01_PZ_TOT_ASSETS_BOY,F9_01_PZ_TOT_EXP_CURR,F9_01_PZ_TOT_LIAB_BOY,F9_01_PZ_TOT_REV_CURR,F9_03_PC_PGMSVC_SIGNIF_CHG,F9_03_PC_PGMSVC_SIGNIF_NEW,F9_03_PC_PROG_SVC_ACC_1_CODE,F9_03_PC_PROG_SVC_ACC_1_DESC,F9_03_PC_PROG_SVC_ACC_1_EXP,F9_03_PC_PROG_SVC_ACC_1_GRNT,F9_03_PC_PROG_SVC_ACC_1_REV,F9_03_PC_PROG_SVC_ACC_2_CODE,F9_03_PC_PROG_SVC_ACC_2_DESC,F9_03_PC_PROG_SVC_ACC_2_EXP,F9_03_PC_PROG_SVC_ACC_2_GRNT,F9_03_PC_PROG_SVC_ACC_2_REV,F9_03_PC_PROG_SVC_ACC_3_CODE,F9_03_PC_PROG_SVC_ACC_3_DESC,F9_03_PC_PROG_SVC_ACC_3_EXP,F9_03_PC_PROG_SVC_ACC_3_GRNT,F9_03_PC_PROG_SVC_ACC_3_REV,F9_03_PC_TOT_OTH_PROG_SVC_EXP,F9_03_PC_TOT_OTH_PROG_SVC_GRNT,F9_03_PC_TOT_OTH_PROG_SVC_REV,F9_03_PC_TOT_PROG_SVC_EXPENSE,F9_03_PZ_MISSION_DESCRIPTION,F9_03_PZ_SCHEDULE_O_PART3,F9_04_PC_ACTVITIES_VIA_PARTNER,F9_04_PC_CONTROLLED_ENTITY,F9_04_PC_DISREGARDED_ENTITY,F9_04_PC_EXCESS_BENEFIT_TRANS,F9_04_PC_FR_EVENT_INC_GT_15K,F9_04_PC_GAMING_INC_GT_15K,F9_04_PC_LOBBYING_ACTIVITIES,F9_04_PC_POLITICAL_ACTIVITIES,F9_04_PC_PRIOR_EXCESS_BEN_TRAN,F9_04_PC_PROF_FR_EXP_GT_15K,F9_04_PC_RELATED_ENTITY,F9_04_PC_TRANS_TO_CNTRLD_ENT,F9_04_PC_TRANS_WITH_CNTRLD_ENT,F9_05_EXP_SCHED_O_X,F9_05_PC_NUMBER_EMPLOYEES_W3,F9_05_PC_NUMBER_FORMS_1096,F9_05_PC_UNRELATED_BUS_INCOME,F9_06_EXP_SCHED_O_X,F9_06_PC_990_PROVIDED_GOV_BODY,F9_06_PC_ANNUAL_DISC_COVRD_PERS,F9_06_PC_CEO_COMPENSTN_PROCESS,F9_06_PC_CHANGES_ORGANIZING_DOCS,F9_06_PC_CONFLICT_OF_INTEREST,F9_06_PC_DECISIONS_SUBJ_APPROVAL,F9_06_PC_DELEGATION_MGT_DUTIES,F9_06_PC_DELEGATION_OF_MGT,F9_06_PC_DOCUMENT_RET_POLICY,F9_06_PC_ELECTION_BOARD_MEMBERS,F9_06_PC_FAMILY_OR_BUSINESS_REL,F9_06_PC_FORM_AVAIL_OWN_WEBSITE,F9_06_PC_FORM_UPON_REQUEST,F9_06_PC_JOINT_VENTURE_INVESTMNT,F9_06_PC_JOINT_VENTURE_POLICY,F9_06_PC_LOCAL_CHAPTERS,F9_06_PC_MATERIAL_DIVERSION,F9_06_PC_MEMBERS_OR_STOCKHOLDERS,F9_06_PC_MINUTES_COMMITTEES,F9_06_PC_MINUTES_GOVERNING_BODY,F9_06_PC_MONITORING_OF_COI_POLICY,F9_06_PC_NUM_IND_VOTING_MEMBERS,F9_06_PC_NUM_VOTING_GOV_MEMBERS,F9_06_PC_OFFICER_MAILING_ADDRESS,F9_06_PC_OTHER_COMPENSTN_PROCESS,F9_06_PC_OTHER_WEBSITE,F9_06_PC_OWN_WEBSITE,F9_06_PC_POLICIES_GOVERN_CHAPTER,F9_06_PC_STATES_WHERE_RET_FILED,F9_

# Change Data Types

In [15]:
new_variables_df['python_data_type'].value_counts()

python_data_type
Int64       262
string       24
DateTime      2
Name: count, dtype: int64

### String variables

In [16]:
string_vars = new_variables_df[new_variables_df['python_data_type']=='string']['variable_name_new'].tolist()
print(len(string_vars))
print(string_vars)

24
['F9_00_HD_CTRY_OF_DOMICILE', 'F9_00_HD_FILER_ADDR_US_L1', 'F9_00_HD_FILER_ADDR_US_L2', 'F9_00_HD_FILER_CITY_US', 'F9_00_HD_FILER_COUNTRY_FRGN', 'F9_00_HD_FILER_STATE_US', 'F9_00_HD_FILER_ZIP_US', 'F9_00_HD_GROSS_EXEMPT_NUM', 'F9_00_HD_PRIN_OFF_NAME', 'F9_00_HD_SIGNING_OFFICER_SIGNTR', 'F9_00_HD_SPECIAL_CONDITION_DESC', 'F9_00_HD_STATE_OF_DOMICILE', 'F9_00_HD_TAX_PER_BEGIN', 'F9_00_HD_TAX_PER_END', 'F9_00_HD_TYPE_ORG_OTHER_DESC', 'F9_00_HD_WEBSITE', 'F9_01_PZ_ORGANIZATIONAL_MISSION', 'F9_03_PC_PROG_SVC_ACC_1_DESC', 'F9_03_PC_PROG_SVC_ACC_2_DESC', 'F9_03_PC_PROG_SVC_ACC_3_DESC', 'F9_03_PZ_MISSION_DESCRIPTION', 'F9_06_PC_STATES_WHERE_RET_FILED', 'F9_12_PC_ACCTG_METHOD_OTHER', 'TaxPeriod']


In [17]:
string_vars.remove('TaxPeriod')

In [18]:
#THIS VARIABLE WAS float64
string_vars.remove('F9_00_HD_FILER_ADDR_US_L2')

In [19]:
df[string_vars].dtypes

F9_00_HD_CTRY_OF_DOMICILE          object
F9_00_HD_FILER_ADDR_US_L1          object
F9_00_HD_FILER_CITY_US             object
F9_00_HD_FILER_COUNTRY_FRGN        object
F9_00_HD_FILER_STATE_US            object
F9_00_HD_FILER_ZIP_US              object
F9_00_HD_GROSS_EXEMPT_NUM          object
F9_00_HD_PRIN_OFF_NAME             object
F9_00_HD_SIGNING_OFFICER_SIGNTR    object
F9_00_HD_SPECIAL_CONDITION_DESC    object
F9_00_HD_STATE_OF_DOMICILE         object
F9_00_HD_TAX_PER_BEGIN             object
F9_00_HD_TAX_PER_END               object
F9_00_HD_TYPE_ORG_OTHER_DESC       object
F9_00_HD_WEBSITE                   object
F9_01_PZ_ORGANIZATIONAL_MISSION    object
F9_03_PC_PROG_SVC_ACC_1_DESC       object
F9_03_PC_PROG_SVC_ACC_2_DESC       object
F9_03_PC_PROG_SVC_ACC_3_DESC       object
F9_03_PZ_MISSION_DESCRIPTION       object
F9_06_PC_STATES_WHERE_RET_FILED    object
F9_12_PC_ACCTG_METHOD_OTHER        object
dtype: object

<br>The above variables are already in *string* format so they can be left alone.

### *DateTime* columns
There are 2 *datetime* columns. We will process each in turn.

In [20]:
DateTime_vars = new_variables_df[new_variables_df['python_data_type']=='DateTime']['variable_name_new'].tolist()
print(len(DateTime_vars))
print(DateTime_vars)

2
['F9_00_HD_BUILD_TIME_STAMP', 'F9_00_HD_TIME_STAMP']


In [21]:
df[DateTime_vars].dtypes

F9_00_HD_BUILD_TIME_STAMP    object
F9_00_HD_TIME_STAMP          object
dtype: object

In [22]:
df[DateTime_vars].sample(5)

,F9_00_HD_BUILD_TIME_STAMP,F9_00_HD_TIME_STAMP
1563444,2019-02-21 02:37:17Z,2019-03-16T07:34:22-07:00
2095050,2021-01-29 14:40:06Z,2020-11-17T08:13:43-06:00
2016786,2021-01-29 14:40:06Z,2020-02-21T15:56:22-06:00
717255,2016-03-07 17:11:31Z,2015-09-28T14:08:55-05:00
1753433,2020-03-31 21:24:44Z,2019-11-01T14:46:41-05:00


##### F9_00_HD_BUILD_TIME_STAMP

In [23]:
pd.to_datetime(df.sample(10)['F9_00_HD_BUILD_TIME_STAMP'])

1784255   2020-03-31 21:24:44+00:00
3263460   2024-10-15 13:58:12+00:00
1771433   2020-03-31 21:24:44+00:00
1276081   2017-02-10 21:41:12+00:00
3060024   2023-04-26 12:10:37+00:00
3342595   2024-10-15 13:58:12+00:00
3255894   2023-04-26 12:10:37+00:00
2435905   2022-09-23 18:48:47+00:00
300198    2016-03-07 17:11:31+00:00
1988560   2021-01-29 14:40:06+00:00
Name: F9_00_HD_BUILD_TIME_STAMP, dtype: datetime64[ns, UTC]

In [24]:
print(len(df[df['F9_00_HD_BUILD_TIME_STAMP'].isnull()]))

0


In [25]:
%%time
df['F9_00_HD_BUILD_TIME_STAMP'] = pd.to_datetime(df['F9_00_HD_BUILD_TIME_STAMP'])

CPU times: total: 391 ms
Wall time: 450 ms


In [26]:
df.sample(5)['F9_00_HD_BUILD_TIME_STAMP']

1417650   2018-06-14 16:35:46+00:00
838401    2016-04-25 22:37:26+00:00
2855860   2023-04-26 12:10:37+00:00
1272961   2017-02-10 21:41:12+00:00
295587    2016-02-25 16:41:14+00:00
Name: F9_00_HD_BUILD_TIME_STAMP, dtype: datetime64[ns, UTC]

In [27]:
df['F9_00_HD_BUILD_TIME_STAMP'].min()

Timestamp('2015-11-30 17:44:51+0000', tz='UTC')

In [28]:
df['F9_00_HD_BUILD_TIME_STAMP'].max()

Timestamp('2025-03-06 01:10:19+0000', tz='UTC')

##### F9_00_HD_TIME_STAMP

In [29]:
df.sample(5)['F9_00_HD_TIME_STAMP']

2467157    2021-09-26T16:03:22-07:00
985337     2016-09-09T15:41:35-05:00
2490646    2021-12-11T09:40:05-06:00
279914     2013-02-05T09:48:40-06:00
1438826    2018-05-15T14:03:00-06:00
Name: F9_00_HD_TIME_STAMP, dtype: object

In [30]:
print(len(df[df['F9_00_HD_TIME_STAMP'].isnull()]))

0


In [45]:
%%time
#df['F9_00_HD_TIME_STAMP'] = pd.to_datetime(df['F9_00_HD_TIME_STAMP'])
df['F9_00_HD_TIME_STAMP'] = pd.to_datetime(df['F9_00_HD_TIME_STAMP'], utc=True)

CPU times: total: 8.23 s
Wall time: 8.64 s


<br>Function to deal with missing values if needed -- See *IRS 990 e-File Preparer Data -- (9n) -- Schedule O - Number of Words and Time Delta Variables.ipynb*

In [34]:
#def timefunc(x):
#    if pd.isnull(x):
#        return np.nan
#    else: 
#        return pd.to_datetime(x)        

In [279]:
#%%time
#df['F9_00_HD_TIME_STAMP_dt'] = df['F9_00_HD_TIME_STAMP'][:].apply(timefunc)
###df['F9_09_PC_TOTAL_FUNC_EXPENSES'] = df['F9_09_PC_TOTAL_FUNC_EXPENSES'].astype('float')
#df[DateTime_vars][:6]

,F9_00_HD_BUILD_TIME_STAMP,F9_00_HD_TIME_STAMP
0,2016-02-24 21:20:13+00:00,2011-11-09 06:41:09-06:00
1,2016-02-24 21:20:13+00:00,2011-11-09 07:32:06-08:00
2,2016-02-24 21:20:13+00:00,2011-11-09 07:33:03-08:00
3,2016-02-24 21:20:13+00:00,2011-11-09 07:54:44-08:00
4,2016-02-24 21:20:13+00:00,2011-11-09 10:05:52-06:00
5,2016-02-24 21:20:13+00:00,2011-11-09 08:42:28-08:00


In [35]:
df.sample(5)['F9_00_HD_TIME_STAMP']

2101424    2020-11-16 17:06:25-06:00
1696300    2019-09-23 09:20:52-07:00
2952923    2023-09-25 05:42:39-07:00
2875065    2023-06-16 19:03:45+00:00
2030012    2021-05-13 14:37:33-05:00
Name: F9_00_HD_TIME_STAMP, dtype: object

In [36]:
for index, row in df[:5].iterrows():
    print(type(row['F9_00_HD_BUILD_TIME_STAMP']), type(row['F9_00_HD_TIME_STAMP']))
    print(row['F9_00_HD_BUILD_TIME_STAMP'].year)
    print(row['F9_00_HD_TIME_STAMP'].year)
    #print(row['F9_00_HD_TIME_STAMP_dt'].year)    

<class 'pandas._libs.tslibs.timestamps.Timestamp'> <class 'pandas._libs.tslibs.timestamps.Timestamp'>
2016
2011
<class 'pandas._libs.tslibs.timestamps.Timestamp'> <class 'pandas._libs.tslibs.timestamps.Timestamp'>
2016
2011
<class 'pandas._libs.tslibs.timestamps.Timestamp'> <class 'pandas._libs.tslibs.timestamps.Timestamp'>
2016
2011
<class 'pandas._libs.tslibs.timestamps.Timestamp'> <class 'pandas._libs.tslibs.timestamps.Timestamp'>
2016
2011
<class 'pandas._libs.tslibs.timestamps.Timestamp'> <class 'pandas._libs.tslibs.timestamps.Timestamp'>
2016
2011


In [37]:
df['F9_00_HD_TIME_STAMP'].min()

Timestamp('2002-01-01 03:19:19-0500', tz='UTC-05:00')

In [38]:
df['F9_00_HD_TIME_STAMP'].max()

Timestamp('2025-02-20 23:50:42-0500', tz='UTC-05:00')

In [40]:
%%time
df['F9_00_HD_TIME_STAMP_yr'] = df['F9_00_HD_TIME_STAMP'].apply(lambda x: str(x)[:4])
print(df['F9_00_HD_TIME_STAMP_yr'].value_counts().sort_index(), '\n')
df[['F9_00_HD_TIME_STAMP_yr']][:2]

F9_00_HD_TIME_STAMP_yr
2002         1
2009       183
2010      1333
2011    117559
2012    142315
2013    175554
2014    193702
2015    211775
2016    228991
2017    239015
2018    252071
2019    261282
2020    273462
2021    305355
2022    331136
2023    347755
2024    358034
2025     29485
Name: count, dtype: int64 

CPU times: total: 14.2 s
Wall time: 14.8 s


,F9_00_HD_TIME_STAMP_yr
0,2011
1,2011


In [41]:
df[df['F9_00_HD_TIME_STAMP_yr']=='2002'][['EIN', 'fiscal_year', '501c3', 'F9_00_HD_BUILD_TIME_STAMP',
                                          'F9_00_HD_TIME_STAMP', 'F9_00_HD_TIME_STAMP_yr']]

,EIN,fiscal_year,501c3,F9_00_HD_BUILD_TIME_STAMP,F9_00_HD_TIME_STAMP,F9_00_HD_TIME_STAMP_yr
234090,010838477,None,1,2016-02-24 21:20:13+00:00,2002-01-01 03:19:19-05:00,2002


In [46]:
df[DateTime_vars].dtypes

F9_00_HD_BUILD_TIME_STAMP    datetime64[ns, UTC]
F9_00_HD_TIME_STAMP          datetime64[ns, UTC]
dtype: object

<br>
*NOTE:* There is one row with a time stamp <2009

### *Int64* columns

In [47]:
new_variables_df[new_variables_df['variable_name_new'].isin(['F9_09_EXP_OTH_OTH_TOT', 'F9_09_EXP_OTH_TOT'])]

,variable_name_new,original_names,python_data_type
190,F9_09_EXP_OTH_OTH_TOT,"[AllOtherExpensesGrp, AllOtherExpenses]",Int64
191,F9_09_EXP_OTH_TOT,"[OtherExpenses, OtherExpensesGrp]",Int64


In [46]:
#new_variables_df = new_variables_df.drop([186,187])
#new_variables_df = new_variables_df.drop([190,191])

In [48]:
variables_to_drop = ['F9_09_EXP_OTH_OTH_TOT', 'F9_09_EXP_OTH_TOT']
print(len(new_variables_df))
new_variables_df = new_variables_df[~new_variables_df['variable_name_new'].isin(variables_to_drop)]
print(len(new_variables_df))

288
286


In [49]:
Int64_vars = new_variables_df[new_variables_df['python_data_type']=='Int64']['variable_name_new'].tolist()
print(len(Int64_vars))
print(Int64_vars)

260
['F9_00_HD_ADDR_CHANGE', 'F9_00_HD_AMENDED_RETURN', 'F9_00_HD_EXEMPT_STATUS_4847A1', 'F9_00_HD_EXEMPT_STATUS_501C', 'F9_00_HD_EXEMPT_STATUS_501C3', 'F9_00_HD_FINAL_RETURN', 'F9_00_HD_GROSS_RCPT', 'F9_00_HD_GROUP_RETURN', 'F9_00_HD_INCLUDES_SUBORD_ORGS', 'F9_00_HD_INITIAL_RETURN', 'F9_00_HD_TAX_YEAR', 'F9_00_HD_TYPE_ORG_ASSOCIATION', 'F9_00_HD_TYPE_ORG_CORP', 'F9_00_HD_TYPE_ORG_OTHER', 'F9_00_HD_TYPE_ORG_TRUST', 'F9_00_HD_YEAR_FORMED', 'F9_01_PC_BEN_PAID_MEMB_PRIOR', 'F9_01_PC_CONTR_GRANTS_CURR', 'F9_01_PC_CONTR_GRANTS_PRIOR', 'F9_01_PC_GRANTS_PRIOR', 'F9_01_PC_INDEP_VOTING_MEMB', 'F9_01_PC_INVEST_INCOME_PRIOR', 'F9_01_PC_NET_ASSETS_BOY', 'F9_01_PC_OTHER_EXPENSE_PRIOR', 'F9_01_PC_OTHER_REV_PRIOR', 'F9_01_PC_PROF_FUNDRISING_EXP_CURR', 'F9_01_PC_PROF_FUNDRISING_EXP_PRIOR', 'F9_01_PC_PROG_SERVICE_REV_PRIOR', 'F9_01_PC_REV_LESS_EXP_CURR', 'F9_01_PC_REV_LESS_EXP_PRIOR', 'F9_01_PC_TERMINATION_CONTRACTION', 'F9_01_PC_TOT_ASSETS_EOY', 'F9_01_PC_TOT_EXP_PRIOR', 'F9_01_PC_TOT_FNDR_EXP_CURR', 

In [50]:
df[Int64_vars].dtypes[:50]

F9_00_HD_ADDR_CHANGE                   int64
F9_00_HD_AMENDED_RETURN                int64
F9_00_HD_EXEMPT_STATUS_4847A1          int64
F9_00_HD_EXEMPT_STATUS_501C           object
F9_00_HD_EXEMPT_STATUS_501C3           int64
F9_00_HD_FINAL_RETURN                  int64
F9_00_HD_GROSS_RCPT                   object
F9_00_HD_GROUP_RETURN                  int64
F9_00_HD_INCLUDES_SUBORD_ORGS          int64
F9_00_HD_INITIAL_RETURN                int64
F9_00_HD_TAX_YEAR                     object
F9_00_HD_TYPE_ORG_ASSOCIATION          int64
F9_00_HD_TYPE_ORG_CORP                 int64
F9_00_HD_TYPE_ORG_OTHER                int64
F9_00_HD_TYPE_ORG_TRUST                int64
F9_00_HD_YEAR_FORMED                  object
F9_01_PC_BEN_PAID_MEMB_PRIOR          object
F9_01_PC_CONTR_GRANTS_CURR            object
F9_01_PC_CONTR_GRANTS_PRIOR           object
F9_01_PC_GRANTS_PRIOR                 object
F9_01_PC_INDEP_VOTING_MEMB            object
F9_01_PC_INVEST_INCOME_PRIOR          object
F9_01_PC_N

<br>These two variables are ``float64`` instead of ``obj`` -- they must all be missing ``['F9_03_PC_PROG_SVC_ACC_2_CODE', 'F9_03_PC_PROG_SVC_ACC_3_CODE']``

In [51]:
df[Int64_vars].dtypes[50:100]

F9_01_PZ_TOT_ASSETS_BOY             object
F9_01_PZ_TOT_EXP_CURR               object
F9_01_PZ_TOT_LIAB_BOY               object
F9_01_PZ_TOT_REV_CURR               object
F9_03_PC_PGMSVC_SIGNIF_CHG           int64
F9_03_PC_PGMSVC_SIGNIF_NEW           int64
F9_03_PC_PROG_SVC_ACC_1_CODE        object
F9_03_PC_PROG_SVC_ACC_1_EXP         object
F9_03_PC_PROG_SVC_ACC_1_GRNT        object
F9_03_PC_PROG_SVC_ACC_1_REV         object
F9_03_PC_PROG_SVC_ACC_2_CODE        object
F9_03_PC_PROG_SVC_ACC_2_EXP         object
F9_03_PC_PROG_SVC_ACC_2_GRNT        object
F9_03_PC_PROG_SVC_ACC_2_REV         object
F9_03_PC_PROG_SVC_ACC_3_CODE        object
F9_03_PC_PROG_SVC_ACC_3_EXP         object
F9_03_PC_PROG_SVC_ACC_3_GRNT        object
F9_03_PC_PROG_SVC_ACC_3_REV         object
F9_03_PC_TOT_OTH_PROG_SVC_EXP       object
F9_03_PC_TOT_OTH_PROG_SVC_GRNT      object
F9_03_PC_TOT_OTH_PROG_SVC_REV       object
F9_03_PC_TOT_PROG_SVC_EXPENSE       object
F9_03_PZ_SCHEDULE_O_PART3            int64
F9_04_PC_AC

In [52]:
df[Int64_vars].dtypes[100:150]

F9_06_PC_ELECTION_BOARD_MEMBERS       int64
F9_06_PC_FAMILY_OR_BUSINESS_REL       int64
F9_06_PC_FORM_AVAIL_OWN_WEBSITE       int64
F9_06_PC_FORM_UPON_REQUEST            int64
F9_06_PC_JOINT_VENTURE_INVESTMNT      int64
F9_06_PC_JOINT_VENTURE_POLICY         int64
F9_06_PC_LOCAL_CHAPTERS               int64
F9_06_PC_MATERIAL_DIVERSION           int64
F9_06_PC_MEMBERS_OR_STOCKHOLDERS      int64
F9_06_PC_MINUTES_COMMITTEES           int64
F9_06_PC_MINUTES_GOVERNING_BODY       int64
F9_06_PC_MONITORING_OF_COI_POLICY     int64
F9_06_PC_NUM_IND_VOTING_MEMBERS      object
F9_06_PC_NUM_VOTING_GOV_MEMBERS      object
F9_06_PC_OFFICER_MAILING_ADDRESS      int64
F9_06_PC_OTHER_COMPENSTN_PROCESS      int64
F9_06_PC_OTHER_WEBSITE                int64
F9_06_PC_OWN_WEBSITE                  int64
F9_06_PC_POLICIES_GOVERN_CHAPTER      int64
F9_06_PC_WHISTLEBLOWER_POLICY         int64
F9_07_EXP_SCHED_O_X                   int64
F9_07_PC_COMPENSATION_OTHER_SRCE      int64
F9_07_PC_FORMER_OFFICER_LISTED  

In [53]:
df[Int64_vars].dtypes[150:200]

F9_09_EXP_AD_PROMO_TOT              object
F9_09_EXP_BENF_PAID_MEMB_TOT        object
F9_09_EXP_CONF_MEETING_TOT          object
F9_09_EXP_DEPREC_FUNDR              object
F9_09_EXP_DEPREC_MAG                object
F9_09_EXP_DEPREC_PROG               object
F9_09_EXP_DEPREC_TOT                object
F9_09_EXP_GRANT_FRGN_TOT            object
F9_09_EXP_GRANT_INDIV_DMSTC_TOT     object
F9_09_EXP_GRANT_ORG_DMSTC_TOT       object
F9_09_EXP_INFO_TECH_TOT             object
F9_09_EXP_INSURANCE_TOT             object
F9_09_EXP_INTEREST_TOT              object
F9_09_EXP_JOINT_COSTS_TOT           object
F9_09_EXP_OCCUPANCY_TOT             object
F9_09_EXP_OFFICE_TOT                object
F9_09_EXP_ROY_TOT                   object
F9_09_EXP_SCHED_O_X                  int64
F9_09_EXP_TRAVEL_ENTRTNMNT_TOT      object
F9_09_EXP_TRAVEL_TOT                object
F9_09_PC_COMP_DISQUAL_FUNDRAISE     object
F9_09_PC_COMP_DISQUAL_MGMT          object
F9_09_PC_COMP_DISQUAL_PROG_SVCE     object
F9_09_PC_CO

In [52]:
df[Int64_vars].dtypes[200:250]

F9_09_PC_PENSION_CONT_PROG_SVCE     object
F9_09_PC_PENSION_CONT_TOTAL         object
F9_09_PC_TOTAL_FUNC_EXPENSES        object
F9_09_PC_TOTAL_FUNDRAISE_EXPENSE    object
F9_09_PC_TOTAL_MGMT_EXPENSE         object
F9_09_PC_TOTAL_PROG_SVCE_EXPENSE    object
F9_10_ASSETS_ACC_NET_EOY            object
F9_10_ASSETS_EXP_PREPAID_EOY        object
F9_10_ASSETS_INTANGIB_EOY           object
F9_10_ASSETS_INVENT_SALE_EOY        object
F9_10_ASSETS_LESS_DEPREC_EOY        object
F9_10_ASSETS_LOANS_DISQUAL_EOY      object
F9_10_ASSETS_NOTES_LOANS_NET_EOY    object
F9_10_ASSETS_OTH_EOY                object
F9_10_ASSETS_PLEDGES_NET_EOY        object
F9_10_LIAB_ACC_PAYABLE_EOY          object
F9_10_LIAB_GRANTS_PAYABLE_EOY       object
F9_10_LIAB_LOANS_OFF_EOY            object
F9_10_LIAB_REV_DEFERRED_EOY         object
F9_10_NAFB_RESTRICT_PERM_EOY        object
F9_10_NAFB_RESTRICT_TEMP_EOY        object
F9_10_NAFB_UNRESTRICT_EOY           object
F9_10_PC_BOND_LIABILITY_EOY         object
F9_10_PC_CA

In [54]:
df[Int64_vars].dtypes[250:]

F9_11_SCHED_O_X                      int64
F9_12_PC_ACCNT_COMPILE_OR_REVIEW     int64
F9_12_PC_ACCTG_METHOD_ACCRUAL        int64
F9_12_PC_ACCTG_METHOD_CASH           int64
F9_12_PC_AUDIT_COMMITTEE             int64
F9_12_PC_FED_GRNT_AUDIT_PERFORMD     int64
F9_12_PC_FED_GRNT_AUDIT_REQUIRED     int64
F9_12_PC_FINCL_STMTS_AUDITED         int64
F9_12_SCHED_O_X                      int64
number_of_other_prog_svces          object
dtype: object

In [58]:
# Step 1: Get the variable names expected to be Int64
#Int64_vars = new_variables_df[new_variables_df['python_data_type']=='Int64']['variable_name_new'].tolist()

# Step 2: Filter to keep only those that are currently of dtype 'object' in df
#not_yet_converted = [var for var in Int64_vars if df[var].dtype == 'object']
#print(len(not_yet_converted))

# Optional: turn it into a DataFrame like the original Int64_vars
#not_yet_converted_df = new_variables_df[new_variables_df['variable_name_new'].isin(not_yet_converted)]

Int64_vars = [var for var in Int64_vars if df[var].dtype == 'object']
print(len(Int64_vars))

182


In [59]:
df[Int64_vars].dtypes[:20]

F9_00_HD_EXEMPT_STATUS_501C           object
F9_00_HD_GROSS_RCPT                   object
F9_00_HD_TAX_YEAR                     object
F9_00_HD_YEAR_FORMED                  object
F9_01_PC_BEN_PAID_MEMB_PRIOR          object
F9_01_PC_CONTR_GRANTS_CURR            object
F9_01_PC_CONTR_GRANTS_PRIOR           object
F9_01_PC_GRANTS_PRIOR                 object
F9_01_PC_INDEP_VOTING_MEMB            object
F9_01_PC_INVEST_INCOME_PRIOR          object
F9_01_PC_NET_ASSETS_BOY               object
F9_01_PC_OTHER_EXPENSE_PRIOR          object
F9_01_PC_OTHER_REV_PRIOR              object
F9_01_PC_PROF_FUNDRISING_EXP_CURR     object
F9_01_PC_PROF_FUNDRISING_EXP_PRIOR    object
F9_01_PC_PROG_SERVICE_REV_PRIOR       object
F9_01_PC_REV_LESS_EXP_CURR            object
F9_01_PC_REV_LESS_EXP_PRIOR           object
F9_01_PC_TOT_ASSETS_EOY               object
F9_01_PC_TOT_EXP_PRIOR                object
dtype: object

##### This approach doesn't work:
- df['F9_00_HD_EXEMPT_STATUS_501C'] = df['F9_00_HD_EXEMPT_STATUS_501C'].astype('Int64')

There are problems converting some variables to 'Int64', with the following error message:
    - TypeError: object cannot be converted to an IntegerDtype

Instead, use the following one-liner -- it chooses whether to convert to 'Int64' or 'float'    

In [60]:
len(Int64_vars)

182

<br>I am guessing the pd.to_numeric issue is due to *number_of_other_prog_svces*, which has a value of, for example:

{'Desc': 'THOMAS BRIDGE WATER CORPORATION PROVIDES WATER SERVICE TO APPROXIMATELY 1700 WATER CUSTOMERS WHICH WOULD NOT HAVE A CLEAN AND ADEQUATE WATER SUPPLY IF NOT FOR THE COMPANY.'}


In [61]:
df[Int64_vars].sample(5)

,F9_00_HD_EXEMPT_STATUS_501C,F9_00_HD_GROSS_RCPT,F9_00_HD_TAX_YEAR,F9_00_HD_YEAR_FORMED,F9_01_PC_BEN_PAID_MEMB_PRIOR,F9_01_PC_CONTR_GRANTS_CURR,F9_01_PC_CONTR_GRANTS_PRIOR,F9_01_PC_GRANTS_PRIOR,F9_01_PC_INDEP_VOTING_MEMB,F9_01_PC_INVEST_INCOME_PRIOR,F9_01_PC_NET_ASSETS_BOY,F9_01_PC_OTHER_EXPENSE_PRIOR,F9_01_PC_OTHER_REV_PRIOR,F9_01_PC_PROF_FUNDRISING_EXP_CURR,F9_01_PC_PROF_FUNDRISING_EXP_PRIOR,F9_01_PC_PROG_SERVICE_REV_PRIOR,F9_01_PC_REV_LESS_EXP_CURR,F9_01_PC_REV_LESS_EXP_PRIOR,F9_01_PC_TOT_ASSETS_EOY,F9_01_PC_TOT_EXP_PRIOR,F9_01_PC_TOT_FNDR_EXP_CURR,F9_01_PC_TOT_INDIV_EMPLOYED,F9_01_PC_TOT_INDIV_VOLUNTEERS,F9_01_PC_TOT_LIABILITIES_EOY,F9_01_PC_TOT_REVENUE_PRIOR,F9_01_PC_TOT_UBI_GROSS,F9_01_PC_TOT_UBI_NET,F9_01_PC_VOTING_MEMB_GOV_BODY,F9_01_PZ_BEN_PAID_TO_MEMB_CURR,F9_01_PZ_GRANTS_PAID_CURR,F9_01_PZ_INVEST_INCOME_CURR,F9_01_PZ_NAFB_EOY,F9_01_PZ_OTHER_EXPENSE_CURR,F9_01_PZ_OTHER_REV_CURR,F9_01_PZ_PROG_SERVICE_REV_CURR,F9_01_PZ_SALARIES_CURR,F9_01_PZ_SALARIES_PRIOR,F9_01_PZ_TOT_ASSETS_BOY,F9_01_PZ_TOT_EXP_CURR,F9_01_PZ_TOT_LIAB_BOY,F9_01_PZ_TOT_REV_CURR,F9_03_PC_PROG_SVC_ACC_1_CODE,F9_03_PC_PROG_SVC_ACC_1_EXP,F9_03_PC_PROG_SVC_ACC_1_GRNT,F9_03_PC_PROG_SVC_ACC_1_REV,F9_03_PC_PROG_SVC_ACC_2_CODE,F9_03_PC_PROG_SVC_ACC_2_EXP,F9_03_PC_PROG_SVC_ACC_2_GRNT,F9_03_PC_PROG_SVC_ACC_2_REV,F9_03_PC_PROG_SVC_ACC_3_CODE,F9_03_PC_PROG_SVC_ACC_3_EXP,F9_03_PC_PROG_SVC_ACC_3_GRNT,F9_03_PC_PROG_SVC_ACC_3_REV,F9_03_PC_TOT_OTH_PROG_SVC_EXP,F9_03_PC_TOT_OTH_PROG_SVC_GRNT,F9_03_PC_TOT_OTH_PROG_SVC_REV,F9_03_PC_TOT_PROG_SVC_EXPENSE,F9_05_PC_NUMBER_EMPLOYEES_W3,F9_05_PC_NUMBER_FORMS_1096,F9_06_PC_NUM_IND_VOTING_MEMBERS,F9_06_PC_NUM_VOTING_GOV_MEMBERS,F9_07_PC_NUM_CONTRCTRS_GRTR_100K,F9_07_PC_NUM_INDS_GREATER_100K,F9_07_PC_TOT_OTHER_COMPENSATION,F9_07_PC_TOT_REPRT_COMP_FROM_ORG,F9_07_PC_TOT_REPRT_COMP_RLTD_ORG,F9_08_PC_ALL_OTHER_CONTRIBUTIONS,F9_08_PC_CONTS_REPRTD_FNDRAISNG,F9_08_PC_COST_OF_GOODS_SOLD,F9_08_PC_FEDERATED_CAMPAIGNS,F9_08_PC_FUNDRAISING_DIRECT_EXP,F9_08_PC_FUNDRAISING_EVENTS,F9_08_PC_FUNDRAISING_GROSS_INC,F9_08_PC_GAMING_DIRECT_EXPENSES,F9_08_PC_GAMING_GROSS_INCOME,F9_08_PC_GOVERNMENT_GRANTS,F9_08_PC_GROSS_SALES_INVENTORY,F9_08_PC_MEMBERSHIP_DUES,F9_08_PC_NONCASH_CONTRIBUTIONS,F9_08_PC_PROGRAM_SVCE_REV_TOTAL,F9_08_PC_RELATED_ORGANIZATIONS,F9_08_PC_TOTAL_CONTRIBUTIONS,F9_08_PC_TOTAL_OTHER_REVENUE,F9_08_PC_TOTAL_PROG_SVCE_REVENUE,F9_08_PC_TOTAL_REVENUE,F9_09_EXP_AD_PROMO_TOT,F9_09_EXP_BENF_PAID_MEMB_TOT,F9_09_EXP_CONF_MEETING_TOT,F9_09_EXP_DEPREC_FUNDR,F9_09_EXP_DEPREC_MAG,F9_09_EXP_DEPREC_PROG,F9_09_EXP_DEPREC_TOT,F9_09_EXP_GRANT_FRGN_TOT,F9_09_EXP_GRANT_INDIV_DMSTC_TOT,F9_09_EXP_GRANT_ORG_DMSTC_TOT,F9_09_EXP_INFO_TECH_TOT,F9_09_EXP_INSURANCE_TOT,F9_09_EXP_INTEREST_TOT,F9_09_EXP_JOINT_COSTS_TOT,F9_09_EXP_OCCUPANCY_TOT,F9_09_EXP_OFFICE_TOT,F9_09_EXP_ROY_TOT,F9_09_EXP_TRAVEL_ENTRTNMNT_TOT,F9_09_EXP_TRAVEL_TOT,F9_09_PC_COMP_DISQUAL_FUNDRAISE,F9_09_PC_COMP_DISQUAL_MGMT,F9_09_PC_COMP_DISQUAL_PROG_SVCE,F9_09_PC_COMP_DISQUAL_TOTAL,F9_09_PC_COMP_OFFICERS_FUNDRAISE,F9_09_PC_COMP_OFFICERS_MGMT,F9_09_PC_COMP_OFFICERS_PROG_SVCE,F9_09_PC_COMP_OFFICERS_TOTAL,F9_09_PC_FEES_FOR_SVCE_ACCT_TOT,F9_09_PC_FEES_FOR_SVCE_FR_TOT,F9_09_PC_FEES_FOR_SVCE_INVST_TOT,F9_09_PC_FEES_FOR_SVCE_LEGL_TOT,F9_09_PC_FEES_FOR_SVCE_LOBB_TOT,F9_09_PC_FEES_FOR_SVCE_MGMT_TOT,F9_09_PC_FEES_FOR_SVCE_OTH_TOT,F9_09_PC_OTHER_EMP_BEN_FUNDRAISE,F9_09_PC_OTHER_EMP_BEN_MGMT,F9_09_PC_OTHER_EMP_BEN_PROG_SVCE,F9_09_PC_OTHER_EMP_BEN_TOTAL,F9_09_PC_OTHER_SALARY_FUNDRAISE,F9_09_PC_OTHER_SALARY_MGMT,F9_09_PC_OTHER_SALARY_PROG_SVCE,F9_09_PC_OTHER_SALARY_TOTAL,F9_09_PC_PAYMENT_TO_AFFILIATES,F9_09_PC_PAYROLL_TAX_FUNDRAISE,F9_09_PC_PAYROLL_TAX_MGMT,F9_09_PC_PAYROLL_TAX_PROG_SVCE,F9_09_PC_PAYROLL_TAX_TOTAL,F9_09_PC_PENSION_CONT_FUNDRAISE,F9_09_PC_PENSION_CONT_MGMT,F9_09_PC_PENSION_CONT_PROG_SVCE,F9_09_PC_PENSION_CONT_TOTAL,F9_09_PC_TOTAL_FUNC_EXPENSES,F9_09_PC_TOTAL_FUNDRAISE_EXPENSE,F9_09_PC_TOTAL_MGMT_EXPENSE,F9_09_PC_TOTAL_PROG_SVCE_EXPENSE,F9_10_ASSETS_ACC_NET_EOY,F9_10_ASSETS_EXP_PREPAID_EOY,F9_10_AS

In [62]:
print(len(Int64_vars))
Int64_vars.remove('number_of_other_prog_svces')
print(len(Int64_vars))

182
181


In [57]:
#%%time
#print(len(df), len(df.columns))
#f[Int64_vars] = df[Int64_vars].apply(pd.to_numeric)
#print(len(df), len(df.columns))
#df[:1]

891980 298
891980 298
CPU times: total: 10min 53s
Wall time: 11min 28s


,URL,F9_09_PC_FEES_FOR_SVCE_FR_TOT,F9_00_HD_BUILD_TIME_STAMP,fiscal_year,EIN,BusinessName,BusinessNameControlTxt,PhoneNum,USAddress,InCareOfNm,ForeignAddress,ForeignPhoneNum,F9_00_HD_ADDR_CHANGE,F9_00_HD_AMENDED_RETURN,F9_00_HD_CTRY_OF_DOMICILE,F9_00_HD_EXEMPT_STATUS_4847A1,F9_00_HD_EXEMPT_STATUS_501C,F9_00_HD_EXEMPT_STATUS_501C3,F9_00_HD_FINAL_RETURN,F9_00_HD_GROSS_EXEMPT_NUM,F9_00_HD_GROSS_RCPT,F9_00_HD_GROUP_RETURN,F9_00_HD_INCLUDES_SUBORD_ORGS,F9_00_HD_INITIAL_RETURN,F9_00_HD_PRIN_OFF_NAME,F9_00_HD_SIGNING_OFFICER_SIGNTR,F9_00_HD_SPECIAL_CONDITION_DESC,F9_00_HD_STATE_OF_DOMICILE,F9_00_HD_TAX_PER_BEGIN,F9_00_HD_TAX_PER_END,F9_00_HD_TAX_YEAR,F9_00_HD_TIME_STAMP,F9_00_HD_TYPE_ORG_ASSOCIATION,F9_00_HD_TYPE_ORG_CORP,F9_00_HD_TYPE_ORG_OTHER,F9_00_HD_TYPE_ORG_OTHER_DESC,F9_00_HD_TYPE_ORG_TRUST,F9_00_HD_WEBSITE,F9_00_HD_YEAR_FORMED,F9_01_PC_BEN_PAID_MEMB_PRIOR,F9_01_PC_CONTR_GRANTS_CURR,F9_01_PC_CONTR_GRANTS_PRIOR,F9_01_PC_GRANTS_PRIOR,F9_01_PC_INDEP_VOTING_MEMB,F9_01_PC_INVEST_INCOME_PRIOR,F9_01_PC_NET_ASSETS_BOY,F9_01_PC_OTHER_EXPENSE_PRIOR,F9_01_PC_OTHER_REV_PRIOR,F9_01_PC_PROF_FUNDRISING_EXP_CURR,F9_01_PC_PROF_FUNDRISING_EXP_PRIOR,F9_01_PC_PROG_SERVICE_REV_PRIOR,F9_01_PC_REV_LESS_EXP_CURR,F9_01_PC_REV_LESS_EXP_PRIOR,F9_01_PC_TERMINATION_CONTRACTION,F9_01_PC_TOT_ASSETS_EOY,F9_01_PC_TOT_EXP_PRIOR,F9_01_PC_TOT_FNDR_EXP_CURR,F9_01_PC_TOT_INDIV_EMPLOYED,F9_01_PC_TOT_INDIV_VOLUNTEERS,F9_01_PC_TOT_LIABILITIES_EOY,F9_01_PC_TOT_REVENUE_PRIOR,F9_01_PC_TOT_UBI_GROSS,F9_01_PC_TOT_UBI_NET,F9_01_PC_VOTING_MEMB_GOV_BODY,F9_01_PZ_BEN_PAID_TO_MEMB_CURR,F9_01_PZ_GRANTS_PAID_CURR,F9_01_PZ_INVEST_INCOME_CURR,F9_01_PZ_NAFB_EOY,F9_01_PZ_ORGANIZATIONAL_MISSION,F9_01_PZ_OTHER_EXPENSE_CURR,F9_01_PZ_OTHER_REV_CURR,F9_01_PZ_PROG_SERVICE_REV_CURR,F9_01_PZ_SALARIES_CURR,F9_01_PZ_SALARIES_PRIOR,F9_01_PZ_TOT_ASSETS_BOY,F9_01_PZ_TOT_EXP_CURR,F9_01_PZ_TOT_LIAB_BOY,F9_01_PZ_TOT_REV_CURR,F9_03_PC_PGMSVC_SIGNIF_CHG,F9_03_PC_PGMSVC_SIGNIF_NEW,F9_03_PC_PROG_SVC_ACC_1_CODE,F9_03_PC_PROG_SVC_ACC_1_DESC,F9_03_PC_PROG_SVC_ACC_1_EXP,F9_03_PC_PROG_SVC_ACC_1_GRNT,F9_03_PC_PROG_SVC_ACC_1_REV,F9_03_PC_PROG_SVC_ACC_2_CODE,F9_03_PC_PROG_SVC_ACC_2_DESC,F9_03_PC_PROG_SVC_ACC_2_EXP,F9_03_PC_PROG_SVC_ACC_2_GRNT,F9_03_PC_PROG_SVC_ACC_2_REV,F9_03_PC_PROG_SVC_ACC_3_CODE,F9_03_PC_PROG_SVC_ACC_3_DESC,F9_03_PC_PROG_SVC_ACC_3_EXP,F9_03_PC_PROG_SVC_ACC_3_GRNT,F9_03_PC_PROG_SVC_ACC_3_REV,F9_03_PC_TOT_OTH_PROG_SVC_EXP,F9_03_PC_TOT_OTH_PROG_SVC_GRNT,F9_03_PC_TOT_OTH_PROG_SVC_REV,F9_03_PC_TOT_PROG_SVC_EXPENSE,F9_03_PZ_MISSION_DESCRIPTION,F9_03_PZ_SCHEDULE_O_PART3,F9_04_PC_ACTVITIES_VIA_PARTNER,F9_04_PC_CONTROLLED_ENTITY,F9_04_PC_DISREGARDED_ENTITY,F9_04_PC_EXCESS_BENEFIT_TRANS,F9_04_PC_FR_EVENT_INC_GT_15K,F9_04_PC_GAMING_INC_GT_15K,F9_04_PC_LOBBYING_ACTIVITIES,F9_04_PC_POLITICAL_ACTIVITIES,F9_04_PC_PRIOR_EXCESS_BEN_TRAN,F9_04_PC_PROF_FR_EXP_GT_15K,F9_04_PC_RELATED_ENTITY,F9_04_PC_TRANS_TO_CNTRLD_ENT,F9_04_PC_TRANS_WITH_CNTRLD_ENT,F9_05_EXP_SCHED_O_X,F9_05_PC_NUMBER_EMPLOYEES_W3,F9_05_PC_NUMBER_FORMS_1096,F9_05_PC_UNRELATED_BUS_INCOME,F9_06_EXP_SCHED_O_X,F9_06_PC_990_PROVIDED_GOV_BODY,F9_06_PC_ANNUAL_DISC_COVRD_PERS,F9_06_PC_CEO_COMPENSTN_PROCESS,F9_06_PC_CHANGES_ORGANIZING_DOCS,F9_06_PC_CONFLICT_OF_INTEREST,F9_06_PC_DECISIONS_SUBJ_APPROVAL,F9_06_PC_DELEGATION_MGT_DUTIES,F9_06_PC_DELEGATION_OF_MGT,F9_06_PC_DOCUMENT_RET_POLICY,F9_06_PC_ELECTION_BOARD_MEMBERS,F9_06_PC_FAMILY_OR_BUSINESS_REL,F9_06_PC_FORM_AVAIL_OWN_WEBSITE,F9_06_PC_FORM_UPON_REQUEST,F9_06_PC_JOINT_VENTURE_INVESTMNT,F9_06_PC_JOINT_VENTURE_POLICY,F9_06_PC_LOCAL_CHAPTERS,F9_06_PC_MATERIAL_DIVERSION,F9_06_PC_MEMBERS_OR_STOCKHOLDERS,F9_06_PC_MINUTES_COMMITTEES,F9_06_PC_MINUTES_GOVERNING_BODY,F9_06_PC_MONITORING_OF_COI_POLICY,F9_06_PC_NUM_IND_VOTING_MEMBERS,F9_06_PC_NUM_VOTING_GOV_MEMBERS,F9_06_PC_OFFICER_MAILING_ADDRESS,F9_06_PC_OTHER_COMPENSTN_PROCESS,F9_06_PC_OTHER_WEBSITE,F9_06_PC_OWN_WEBSITE,F9_06_PC_POLICIES_GOVERN_CHAPTER,F9_06_PC_STATES_WHERE_RET_FILED,F9_06_PC_WHISTLEBLOWER_POLICY,F9_07_EXP_SCHED_O_X,F9_07_PC_COMPENSATION_OT

#### Updated way
This concise and efficient version will convert them all safely from 'object' to pandas' nullable 'Int64' type.

✅ Why this works well:
`pd.to_numeric(..., errors='coerce')`: Safely converts strings to numbers, setting invalid entries to `NaN`.

`.astype('Int64')`: Converts to Pandas' nullable integer type (not the NumPy `int64`, which can't store missing values).

In [ ]:
#%%time
#print(len(df), len(df.columns))
#df[Int64_vars] = df[Int64_vars].apply(lambda col: pd.to_numeric(col, errors='coerce').astype('Int64'))
#print(len(df), len(df.columns))
#df[:1]

Version  that logs the number of `NaN` values before and after conversion for each column — so you can see how many values were coerced due to invalid data during the conversion:

In [63]:
%%time
print(len(df), len(df.columns))

for col in Int64_vars:
    before_na = df[col].isna().sum()
    df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')
    after_na = df[col].isna().sum()
    newly_na = after_na - before_na
    print(f"{col}: {newly_na} newly missing (coerced)")

print(len(df), len(df.columns))
df[:1]

3469008 306
F9_00_HD_EXEMPT_STATUS_501C: 0 newly missing (coerced)
F9_00_HD_GROSS_RCPT: 0 newly missing (coerced)
F9_00_HD_TAX_YEAR: 0 newly missing (coerced)
F9_00_HD_YEAR_FORMED: 0 newly missing (coerced)
F9_01_PC_BEN_PAID_MEMB_PRIOR: 0 newly missing (coerced)
F9_01_PC_CONTR_GRANTS_CURR: 0 newly missing (coerced)
F9_01_PC_CONTR_GRANTS_PRIOR: 0 newly missing (coerced)
F9_01_PC_GRANTS_PRIOR: 0 newly missing (coerced)
F9_01_PC_INDEP_VOTING_MEMB: 0 newly missing (coerced)
F9_01_PC_INVEST_INCOME_PRIOR: 0 newly missing (coerced)
F9_01_PC_NET_ASSETS_BOY: 0 newly missing (coerced)
F9_01_PC_OTHER_EXPENSE_PRIOR: 0 newly missing (coerced)
F9_01_PC_OTHER_REV_PRIOR: 0 newly missing (coerced)
F9_01_PC_PROF_FUNDRISING_EXP_CURR: 0 newly missing (coerced)
F9_01_PC_PROF_FUNDRISING_EXP_PRIOR: 0 newly missing (coerced)
F9_01_PC_PROG_SERVICE_REV_PRIOR: 0 newly missing (coerced)
F9_01_PC_REV_LESS_EXP_CURR: 0 newly missing (coerced)
F9_01_PC_REV_LESS_EXP_PRIOR: 0 newly missing (coerced)
F9_01_PC_TOT_ASSETS

F9_10_LIAB_ACC_PAYABLE_EOY: 0 newly missing (coerced)
F9_10_LIAB_GRANTS_PAYABLE_EOY: 0 newly missing (coerced)
F9_10_LIAB_LOANS_OFF_EOY: 0 newly missing (coerced)
F9_10_LIAB_REV_DEFERRED_EOY: 0 newly missing (coerced)
F9_10_NAFB_RESTRICT_PERM_EOY: 0 newly missing (coerced)
F9_10_NAFB_RESTRICT_TEMP_EOY: 0 newly missing (coerced)
F9_10_NAFB_UNRESTRICT_EOY: 0 newly missing (coerced)
F9_10_PC_BOND_LIABILITY_EOY: 0 newly missing (coerced)
F9_10_PC_CASH_NON_INTEREST_BOY: 0 newly missing (coerced)
F9_10_PC_CASH_NON_INTEREST_EOY: 0 newly missing (coerced)
F9_10_PC_ESCROW_LIABILITY_EOY: 0 newly missing (coerced)
F9_10_PC_INVEST_OTHER_SEC_EOY: 0 newly missing (coerced)
F9_10_PC_INVEST_PROG_RELTD_EOY: 0 newly missing (coerced)
F9_10_PC_INVEST_PUB_TRADED_EOY: 0 newly missing (coerced)
F9_10_PC_LAND_BLDG_EQPMT: 0 newly missing (coerced)
F9_10_PC_LAND_BLDG_EQPMT_DEPRCTN: 0 newly missing (coerced)
F9_10_PC_LOANS_FROM_OFFICERS_EOY: 0 newly missing (coerced)
F9_10_PC_OTHER_LIABILITIES_EOY: 0 newly miss

,_id,OrganizationName,URL,DLN,TaxPeriod,F9_09_PC_FEES_FOR_SVCE_FR_TOT,F9_00_HD_BUILD_TIME_STAMP,fiscal_year,EIN,Name,NameControl,Phone,USAddress,ForeignAddress,InCareOfName,BusinessName,BusinessNameControlTxt,PhoneNum,InCareOfNm,ForeignPhoneNum,F9_00_HD_ADDR_CHANGE,F9_00_HD_AMENDED_RETURN,F9_00_HD_CTRY_OF_DOMICILE,F9_00_HD_EXEMPT_STATUS_4847A1,F9_00_HD_EXEMPT_STATUS_501C,F9_00_HD_EXEMPT_STATUS_501C3,F9_00_HD_FINAL_RETURN,F9_00_HD_GROSS_EXEMPT_NUM,F9_00_HD_GROSS_RCPT,F9_00_HD_GROUP_RETURN,F9_00_HD_INCLUDES_SUBORD_ORGS,F9_00_HD_INITIAL_RETURN,F9_00_HD_PRIN_OFF_NAME,F9_00_HD_SIGNING_OFFICER_SIGNTR,F9_00_HD_SPECIAL_CONDITION_DESC,F9_00_HD_STATE_OF_DOMICILE,F9_00_HD_TAX_PER_BEGIN,F9_00_HD_TAX_PER_END,F9_00_HD_TAX_YEAR,F9_00_HD_TIME_STAMP,F9_00_HD_TYPE_ORG_ASSOCIATION,F9_00_HD_TYPE_ORG_CORP,F9_00_HD_TYPE_ORG_OTHER,F9_00_HD_TYPE_ORG_OTHER_DESC,F9_00_HD_TYPE_ORG_TRUST,F9_00_HD_WEBSITE,F9_00_HD_YEAR_FORMED,F9_01_PC_BEN_PAID_MEMB_PRIOR,F9_01_PC_CONTR_GRANTS_CURR,F9_01_PC_CONTR_GRANTS_PRIOR,F9_01_PC_GRANTS_PRIOR,F9_01_PC_INDEP_VOTING_MEMB,F9_01_PC_INVEST_INCOME_PRIOR,F9_01_PC_NET_ASSETS_BOY,F9_01_PC_OTHER_EXPENSE_PRIOR,F9_01_PC_OTHER_REV_PRIOR,F9_01_PC_PROF_FUNDRISING_EXP_CURR,F9_01_PC_PROF_FUNDRISING_EXP_PRIOR,F9_01_PC_PROG_SERVICE_REV_PRIOR,F9_01_PC_REV_LESS_EXP_CURR,F9_01_PC_REV_LESS_EXP_PRIOR,F9_01_PC_TERMINATION_CONTRACTION,F9_01_PC_TOT_ASSETS_EOY,F9_01_PC_TOT_EXP_PRIOR,F9_01_PC_TOT_FNDR_EXP_CURR,F9_01_PC_TOT_INDIV_EMPLOYED,F9_01_PC_TOT_INDIV_VOLUNTEERS,F9_01_PC_TOT_LIABILITIES_EOY,F9_01_PC_TOT_REVENUE_PRIOR,F9_01_PC_TOT_UBI_GROSS,F9_01_PC_TOT_UBI_NET,F9_01_PC_VOTING_MEMB_GOV_BODY,F9_01_PZ_BEN_PAID_TO_MEMB_CURR,F9_01_PZ_GRANTS_PAID_CURR,F9_01_PZ_INVEST_INCOME_CURR,F9_01_PZ_NAFB_EOY,F9_01_PZ_ORGANIZATIONAL_MISSION,F9_01_PZ_OTHER_EXPENSE_CURR,F9_01_PZ_OTHER_REV_CURR,F9_01_PZ_PROG_SERVICE_REV_CURR,F9_01_PZ_SALARIES_CURR,F9_01_PZ_SALARIES_PRIOR,F9_01_PZ_TOT_ASSETS_BOY,F9_01_PZ_TOT_EXP_CURR,F9_01_PZ_TOT_LIAB_BOY,F9_01_PZ_TOT_REV_CURR,F9_03_PC_PGMSVC_SIGNIF_CHG,F9_03_PC_PGMSVC_SIGNIF_NEW,F9_03_PC_PROG_SVC_ACC_1_CODE,F9_03_PC_PROG_SVC_ACC_1_DESC,F9_03_PC_PROG_SVC_ACC_1_EXP,F9_03_PC_PROG_SVC_ACC_1_GRNT,F9_03_PC_PROG_SVC_ACC_1_REV,F9_03_PC_PROG_SVC_ACC_2_CODE,F9_03_PC_PROG_SVC_ACC_2_DESC,F9_03_PC_PROG_SVC_ACC_2_EXP,F9_03_PC_PROG_SVC_ACC_2_GRNT,F9_03_PC_PROG_SVC_ACC_2_REV,F9_03_PC_PROG_SVC_ACC_3_CODE,F9_03_PC_PROG_SVC_ACC_3_DESC,F9_03_PC_PROG_SVC_ACC_3_EXP,F9_03_PC_PROG_SVC_ACC_3_GRNT,F9_03_PC_PROG_SVC_ACC_3_REV,F9_03_PC_TOT_OTH_PROG_SVC_EXP,F9_03_PC_TOT_OTH_PROG_SVC_GRNT,F9_03_PC_TOT_OTH_PROG_SVC_REV,F9_03_PC_TOT_PROG_SVC_EXPENSE,F9_03_PZ_MISSION_DESCRIPTION,F9_03_PZ_SCHEDULE_O_PART3,F9_04_PC_ACTVITIES_VIA_PARTNER,F9_04_PC_CONTROLLED_ENTITY,F9_04_PC_DISREGARDED_ENTITY,F9_04_PC_EXCESS_BENEFIT_TRANS,F9_04_PC_FR_EVENT_INC_GT_15K,F9_04_PC_GAMING_INC_GT_15K,F9_04_PC_LOBBYING_ACTIVITIES,F9_04_PC_POLITICAL_ACTIVITIES,F9_04_PC_PRIOR_EXCESS_BEN_TRAN,F9_04_PC_PROF_FR_EXP_GT_15K,F9_04_PC_RELATED_ENTITY,F9_04_PC_TRANS_TO_CNTRLD_ENT,F9_04_PC_TRANS_WITH_CNTRLD_ENT,F9_05_EXP_SCHED_O_X,F9_05_PC_NUMBER_EMPLOYEES_W3,F9_05_PC_NUMBER_FORMS_1096,F9_05_PC_UNRELATED_BUS_INCOME,F9_06_EXP_SCHED_O_X,F9_06_PC_990_PROVIDED_GOV_BODY,F9_06_PC_ANNUAL_DISC_COVRD_PERS,F9_06_PC_CEO_COMPENSTN_PROCESS,F9_06_PC_CHANGES_ORGANIZING_DOCS,F9_06_PC_CONFLICT_OF_INTEREST,F9_06_PC_DECISIONS_SUBJ_APPROVAL,F9_06_PC_DELEGATION_MGT_DUTIES,F9_06_PC_DELEGATION_OF_MGT,F9_06_PC_DOCUMENT_RET_POLICY,F9_06_PC_ELECTION_BOARD_MEMBERS,F9_06_PC_FAMILY_OR_BUSINESS_REL,F9_06_PC_FORM_AVAIL_OWN_WEBSITE,F9_06_PC_FORM_UPON_REQUEST,F9_06_PC_JOINT_VENTURE_INVESTMNT,F9_06_PC_JOINT_VENTURE_POLICY,F9_06_PC_LOCAL_CHAPTERS,F9_06_PC_MATERIAL_DIVERSION,F9_06_PC_MEMBERS_OR_STOCKHOLDERS,F9_06_PC_MINUTES_COMMITTEES,F9_06_PC_MINUTES_GOVERNING_BODY,F9_06_PC_MONITORING_OF_COI_POLICY,F9_06_PC_NUM_IND_VOTING_MEMBERS,F9_06_PC_NUM_VOTING_GOV_MEMBERS,F9_06_PC_OFFICER_MAILING_ADDRESS,F9_06_PC_OTHER_COMPENSTN_PROCESS,F9_06_PC_OTHER_WEBSITE,F9_06_PC_OWN_WEBSITE,F9_06_PC_POLICIES_GOVERN_CHAPTER,F9_06_PC_STATES_WHERE_RET_FILED,F9_

In [64]:
%%time
df[Int64_vars].dtypes[:25]

CPU times: total: 2.41 s
Wall time: 2.63 s


F9_00_HD_EXEMPT_STATUS_501C           Int64
F9_00_HD_GROSS_RCPT                   Int64
F9_00_HD_TAX_YEAR                     Int64
F9_00_HD_YEAR_FORMED                  Int64
F9_01_PC_BEN_PAID_MEMB_PRIOR          Int64
F9_01_PC_CONTR_GRANTS_CURR            Int64
F9_01_PC_CONTR_GRANTS_PRIOR           Int64
F9_01_PC_GRANTS_PRIOR                 Int64
F9_01_PC_INDEP_VOTING_MEMB            Int64
F9_01_PC_INVEST_INCOME_PRIOR          Int64
F9_01_PC_NET_ASSETS_BOY               Int64
F9_01_PC_OTHER_EXPENSE_PRIOR          Int64
F9_01_PC_OTHER_REV_PRIOR              Int64
F9_01_PC_PROF_FUNDRISING_EXP_CURR     Int64
F9_01_PC_PROF_FUNDRISING_EXP_PRIOR    Int64
F9_01_PC_PROG_SERVICE_REV_PRIOR       Int64
F9_01_PC_REV_LESS_EXP_CURR            Int64
F9_01_PC_REV_LESS_EXP_PRIOR           Int64
F9_01_PC_TOT_ASSETS_EOY               Int64
F9_01_PC_TOT_EXP_PRIOR                Int64
F9_01_PC_TOT_FNDR_EXP_CURR            Int64
F9_01_PC_TOT_INDIV_EMPLOYED           Int64
F9_01_PC_TOT_INDIV_VOLUNTEERS   

#### Check `dtypes`
Quickly check which variables in Int64_vars are not yet of type 'Int64' and count them using:

In [67]:
%%time
# Count how many are not of dtype 'Int64'
not_int64 = df[Int64_vars].dtypes != 'Int64'
not_int64.sum()

CPU times: total: 2.45 s
Wall time: 2.59 s


0

✅ Want to see which ones specifically?

In [68]:
%%time
df[Int64_vars].dtypes[not_int64]

CPU times: total: 2.41 s
Wall time: 2.54 s


Series([], dtype: object)

✅ Alternatively, in one line:

In [69]:
%%time
df[Int64_vars].dtypes[df[Int64_vars].dtypes != 'Int64']

CPU times: total: 4.89 s
Wall time: 5.17 s


Series([], dtype: object)

In [70]:
df[Int64_vars].sample(10)

,F9_00_HD_EXEMPT_STATUS_501C,F9_00_HD_GROSS_RCPT,F9_00_HD_TAX_YEAR,F9_00_HD_YEAR_FORMED,F9_01_PC_BEN_PAID_MEMB_PRIOR,F9_01_PC_CONTR_GRANTS_CURR,F9_01_PC_CONTR_GRANTS_PRIOR,F9_01_PC_GRANTS_PRIOR,F9_01_PC_INDEP_VOTING_MEMB,F9_01_PC_INVEST_INCOME_PRIOR,F9_01_PC_NET_ASSETS_BOY,F9_01_PC_OTHER_EXPENSE_PRIOR,F9_01_PC_OTHER_REV_PRIOR,F9_01_PC_PROF_FUNDRISING_EXP_CURR,F9_01_PC_PROF_FUNDRISING_EXP_PRIOR,F9_01_PC_PROG_SERVICE_REV_PRIOR,F9_01_PC_REV_LESS_EXP_CURR,F9_01_PC_REV_LESS_EXP_PRIOR,F9_01_PC_TOT_ASSETS_EOY,F9_01_PC_TOT_EXP_PRIOR,F9_01_PC_TOT_FNDR_EXP_CURR,F9_01_PC_TOT_INDIV_EMPLOYED,F9_01_PC_TOT_INDIV_VOLUNTEERS,F9_01_PC_TOT_LIABILITIES_EOY,F9_01_PC_TOT_REVENUE_PRIOR,F9_01_PC_TOT_UBI_GROSS,F9_01_PC_TOT_UBI_NET,F9_01_PC_VOTING_MEMB_GOV_BODY,F9_01_PZ_BEN_PAID_TO_MEMB_CURR,F9_01_PZ_GRANTS_PAID_CURR,F9_01_PZ_INVEST_INCOME_CURR,F9_01_PZ_NAFB_EOY,F9_01_PZ_OTHER_EXPENSE_CURR,F9_01_PZ_OTHER_REV_CURR,F9_01_PZ_PROG_SERVICE_REV_CURR,F9_01_PZ_SALARIES_CURR,F9_01_PZ_SALARIES_PRIOR,F9_01_PZ_TOT_ASSETS_BOY,F9_01_PZ_TOT_EXP_CURR,F9_01_PZ_TOT_LIAB_BOY,F9_01_PZ_TOT_REV_CURR,F9_03_PC_PROG_SVC_ACC_1_CODE,F9_03_PC_PROG_SVC_ACC_1_EXP,F9_03_PC_PROG_SVC_ACC_1_GRNT,F9_03_PC_PROG_SVC_ACC_1_REV,F9_03_PC_PROG_SVC_ACC_2_CODE,F9_03_PC_PROG_SVC_ACC_2_EXP,F9_03_PC_PROG_SVC_ACC_2_GRNT,F9_03_PC_PROG_SVC_ACC_2_REV,F9_03_PC_PROG_SVC_ACC_3_CODE,F9_03_PC_PROG_SVC_ACC_3_EXP,F9_03_PC_PROG_SVC_ACC_3_GRNT,F9_03_PC_PROG_SVC_ACC_3_REV,F9_03_PC_TOT_OTH_PROG_SVC_EXP,F9_03_PC_TOT_OTH_PROG_SVC_GRNT,F9_03_PC_TOT_OTH_PROG_SVC_REV,F9_03_PC_TOT_PROG_SVC_EXPENSE,F9_05_PC_NUMBER_EMPLOYEES_W3,F9_05_PC_NUMBER_FORMS_1096,F9_06_PC_NUM_IND_VOTING_MEMBERS,F9_06_PC_NUM_VOTING_GOV_MEMBERS,F9_07_PC_NUM_CONTRCTRS_GRTR_100K,F9_07_PC_NUM_INDS_GREATER_100K,F9_07_PC_TOT_OTHER_COMPENSATION,F9_07_PC_TOT_REPRT_COMP_FROM_ORG,F9_07_PC_TOT_REPRT_COMP_RLTD_ORG,F9_08_PC_ALL_OTHER_CONTRIBUTIONS,F9_08_PC_CONTS_REPRTD_FNDRAISNG,F9_08_PC_COST_OF_GOODS_SOLD,F9_08_PC_FEDERATED_CAMPAIGNS,F9_08_PC_FUNDRAISING_DIRECT_EXP,F9_08_PC_FUNDRAISING_EVENTS,F9_08_PC_FUNDRAISING_GROSS_INC,F9_08_PC_GAMING_DIRECT_EXPENSES,F9_08_PC_GAMING_GROSS_INCOME,F9_08_PC_GOVERNMENT_GRANTS,F9_08_PC_GROSS_SALES_INVENTORY,F9_08_PC_MEMBERSHIP_DUES,F9_08_PC_NONCASH_CONTRIBUTIONS,F9_08_PC_PROGRAM_SVCE_REV_TOTAL,F9_08_PC_RELATED_ORGANIZATIONS,F9_08_PC_TOTAL_CONTRIBUTIONS,F9_08_PC_TOTAL_OTHER_REVENUE,F9_08_PC_TOTAL_PROG_SVCE_REVENUE,F9_08_PC_TOTAL_REVENUE,F9_09_EXP_AD_PROMO_TOT,F9_09_EXP_BENF_PAID_MEMB_TOT,F9_09_EXP_CONF_MEETING_TOT,F9_09_EXP_DEPREC_FUNDR,F9_09_EXP_DEPREC_MAG,F9_09_EXP_DEPREC_PROG,F9_09_EXP_DEPREC_TOT,F9_09_EXP_GRANT_FRGN_TOT,F9_09_EXP_GRANT_INDIV_DMSTC_TOT,F9_09_EXP_GRANT_ORG_DMSTC_TOT,F9_09_EXP_INFO_TECH_TOT,F9_09_EXP_INSURANCE_TOT,F9_09_EXP_INTEREST_TOT,F9_09_EXP_JOINT_COSTS_TOT,F9_09_EXP_OCCUPANCY_TOT,F9_09_EXP_OFFICE_TOT,F9_09_EXP_ROY_TOT,F9_09_EXP_TRAVEL_ENTRTNMNT_TOT,F9_09_EXP_TRAVEL_TOT,F9_09_PC_COMP_DISQUAL_FUNDRAISE,F9_09_PC_COMP_DISQUAL_MGMT,F9_09_PC_COMP_DISQUAL_PROG_SVCE,F9_09_PC_COMP_DISQUAL_TOTAL,F9_09_PC_COMP_OFFICERS_FUNDRAISE,F9_09_PC_COMP_OFFICERS_MGMT,F9_09_PC_COMP_OFFICERS_PROG_SVCE,F9_09_PC_COMP_OFFICERS_TOTAL,F9_09_PC_FEES_FOR_SVCE_ACCT_TOT,F9_09_PC_FEES_FOR_SVCE_FR_TOT,F9_09_PC_FEES_FOR_SVCE_INVST_TOT,F9_09_PC_FEES_FOR_SVCE_LEGL_TOT,F9_09_PC_FEES_FOR_SVCE_LOBB_TOT,F9_09_PC_FEES_FOR_SVCE_MGMT_TOT,F9_09_PC_FEES_FOR_SVCE_OTH_TOT,F9_09_PC_OTHER_EMP_BEN_FUNDRAISE,F9_09_PC_OTHER_EMP_BEN_MGMT,F9_09_PC_OTHER_EMP_BEN_PROG_SVCE,F9_09_PC_OTHER_EMP_BEN_TOTAL,F9_09_PC_OTHER_SALARY_FUNDRAISE,F9_09_PC_OTHER_SALARY_MGMT,F9_09_PC_OTHER_SALARY_PROG_SVCE,F9_09_PC_OTHER_SALARY_TOTAL,F9_09_PC_PAYMENT_TO_AFFILIATES,F9_09_PC_PAYROLL_TAX_FUNDRAISE,F9_09_PC_PAYROLL_TAX_MGMT,F9_09_PC_PAYROLL_TAX_PROG_SVCE,F9_09_PC_PAYROLL_TAX_TOTAL,F9_09_PC_PENSION_CONT_FUNDRAISE,F9_09_PC_PENSION_CONT_MGMT,F9_09_PC_PENSION_CONT_PROG_SVCE,F9_09_PC_PENSION_CONT_TOTAL,F9_09_PC_TOTAL_FUNC_EXPENSES,F9_09_PC_TOTAL_FUNDRAISE_EXPENSE,F9_09_PC_TOTAL_MGMT_EXPENSE,F9_09_PC_TOTAL_PROG_SVCE_EXPENSE,F9_10_ASSETS_ACC_NET_EOY,F9_10_ASSETS_EXP_PREPAID_EOY,F9_10_AS

In [73]:
gc.collect()

710

In [74]:
%%time
df.describe().T

CPU times: total: 43 s
Wall time: 44.6 s


,count,mean,std,min,25%,50%,75%,max
F9_09_PC_FEES_FOR_SVCE_FR_TOT,759063.0,19424.472779,304953.261139,-35000.0,0.0,0.0,0.0,97665783.0
F9_00_HD_ADDR_CHANGE,3469008.0,0.039835,0.195571,0.0,0.0,0.0,0.0,1.0
F9_00_HD_AMENDED_RETURN,3469008.0,0.01192,0.108527,0.0,0.0,0.0,0.0,1.0
F9_00_HD_EXEMPT_STATUS_4847A1,3469008.0,0.000709,0.026626,0.0,0.0,0.0,0.0,1.0
F9_00_HD_EXEMPT_STATUS_501C,857105.0,7.263418,4.007911,2.0,5.0,6.0,8.0,29.0
...,...,...,...,...,...,...,...,...
F9_12_PC_FED_GRNT_AUDIT_PERFORMD,3469008.0,0.085901,0.280217,0.0,0.0,0.0,0.0,1.0
F9_12_PC_FED_GRNT_AUDIT_REQUIRED,3469008.0,0.086664,0.281342,0.0,0.0,0.0,0.0,1.0
F9_12_PC_FINCL_STMTS_AUDITED,3469008.0,0.434902,0.495744,0.0,0.0,0.0,1.0,1.0
F9_12_SCHED_O_X,3469008.0,0.165592,0.371714,0.0,0.0,0.0,0.0,1.0


#### Save DF

In [75]:
len(df)

3469008

In [76]:
def prepare_for_save(df):
    import gc

    # Drop any cached views
    df = df.copy()  # Break reference to any partial evaluation from .head(), etc.

    # Optionally sort or reset if needed
    # df = df.sort_values("some_column")  # Only if relevant
    # df = df.reset_index(drop=True)

    # Trigger garbage collection
    gc.collect()

    print("🧼 DataFrame copied + garbage collected. Ready to save.")
    return df

In [ ]:
# Step 2: Clean up df (especially if you’ve been doing .head(), .sort(), etc.)
#df = prepare_for_save(df)

In [77]:
%%time
import datetime
print ("Current date and time : ", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), '\n')
df.to_feather('D:/all_filings_april_2025_all_controls_combined_parsed_type.feather')

Current date and time :  2025-04-18 21:47:54 

CPU times: total: 1min 7s
Wall time: 45.5 s


In [78]:
%%time
import datetime
print ("Current date and time : ", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), '\n')
df.to_parquet("D:/all_filings_april_2025_all_controls_combined_parsed_type.parquet", engine="pyarrow", compression="snappy", index=False)

Current date and time :  2025-04-18 21:49:41 

CPU times: total: 1min 54s
Wall time: 1min 56s


In [79]:
%%time
import datetime
print ("Current date and time : ", datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), '\n')
df.to_pickle('D:/all_filings_april_2025_all_controls_combined_parsed_type.pkl.gz', compression='gzip')

Current date and time :  2025-04-18 21:53:19 

CPU times: total: 1h 31min 9s
Wall time: 1h 34min 23s
